
## Imports functions needed

In [ ]:

import os
from zipfile import ZipFile 
import csv


## Creates Headers, Links to holder and makes new folder to store cleaned files

In [ ]:
#this is the first row of headers (I pulled them from down below and put them here then removed the print function)
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

In [ ]:
#It saves the directory of the file called ZipFiles which is currently holding the uncleaned zip files 
#and names it as the variable zip_files
#it also creates a temporary directory (Folder) for the cleaned files called temp_clean


zip_files = os.listdir("ZipFiles")

if not os.path.isdir("temp_clean") : # if folder exists
    os.mkdir("temp_clean")           # if not, make it




## File Cleaning and Creating New File

In [ ]:

#read each folder in the zip_files (where the uncleaned folders are stored)

for zip_file in zip_files :
    with ZipFile("ZipFiles/" + zip_file,'r') as my_zip_file : 
    
        files_inside = my_zip_file.namelist()
        for zipped_file in files_inside :
            sniffer = csv.Sniffer() #checks if it has a header

            #opens,unzips, and reads each file in the zipped folder and name this input_file, 
            #the output files names will be the same name but ends in .clean.csv
            
            with my_zip_file.open(zipped_file,'r') as input_file :
            
                
                output_file_name = input_file.name.replace(".csv","_clean.csv")
    
                #open the folder for the new files that was created above and create a new file using the out_file_name
                with open("temp_clean/" + output_file_name,'w') as outfile : 
                    outfile.write(",".join(headers) + "\n") #adds header to the first line of the file since we remove it below
               
            
                    #the following code removes the byte format, removes the quotes, 
                    #and replaces the null characters with blanks for each line in the input file                
                    rows_printed = 0
                    for idx, line in enumerate(input_file) :

                        file_has_header = False

                        dialect = sniffer.sniff(line.decode("utf-8"))
                        line = line.decode("utf-8").strip().split(dialect.delimiter)
                        line = [piece.replace('"','') for piece in line]
                        line = [piece.replace('NULL','') for piece in line]
                        line = [piece.replace('\\N','') for piece in line]
                        
                        #there was an issue with three columns in the dataset that should be one column, 
                        #this finds the line that is too long, combines the three columns, removes the bad column, and inserts the correct column
                        if len(line) != 50:
                            bad_line = line
                            bad_line[5:8] #the issue is in columns 5 through 7
                            new_column = ("".join(bad_line[5:8]))
                            #print(new_column)
                            
                            #delete bad
                            #print(len(bad_line))
                            del bad_line[5:8]
                            #print(bad_line)
                            #print(len(bad_line))
                            
                            #insert good
                            bad_line.insert(5,new_column)
                            #print(bad_line)
                            #print(len(bad_line))                        
                        
                        
                        
                        #if the first row has datetime in it, then the file has a header
                        if idx == 0 :
                            if 'datetime' in line[0] :
                                file_has_header = True
                        
                        #if the first row has a header, then dont print the header but write the rest of the lines to the file
                        if file_has_header and idx == 0 :
                            # don't print line
                            pass
                        else : 
                            outfile.write(",".join(line) + "\n")
                            rows_printed += 1
                               
    
                        
#print("Finished processing all files into clean files") #this way I know it finished
